# Lab. 3-1 Schema Preparation-2

이 노트북에서는 아래 그림의 `3. Table Summarizer` 과정을 수행합니다. 

일반적인 Schema Linking 과정이 테이블 선택 -> 컬럼 선택으로 나눠 진행되는데, 테이블 선택이 잘못되면 후속 과정은 무의미하기 때문에 각 테이블에 대한 충분한 설명을 갖추는 것이 중요합니다.

이 노트북에서는 LLM을 활용해 테이블에 대한 자세한 설명 문서를 갖추는 과정을 시뮬레이션 합니다.

![Intro](../images/text2sql/schema-prep-1.png)


In [1]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [14]:
from libs.common_utils import retry
from botocore.exceptions import ClientError

## Step 0: OpenSearch 환경 설정

In [2]:
import sys
from libs.ssm import parameter_store

pm = parameter_store('us-west-2')
domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)
print(opensearch_domain_endpoint)

https://https://search-rag-hol-1379d7f0-oxnizb4mkve2hr7542ckzzqqs4.us-west-2.es.amazonaws.com


## Step 1: Schema Description 및 Example Queries 로드

In [3]:
import json 
SCHEMA_FILE_PATH = "./chinook_schema.json"
SAMPLE_QUERY_FILE_PATH = "./example_queries_temp.jsonl"

def load_schema(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        schema = json.load(file)
    return schema

def load_queries(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        queries = file.readlines()
    return queries

schema = load_schema(SCHEMA_FILE_PATH)
queries = load_queries(SAMPLE_QUERY_FILE_PATH)

## Step 2: 테이블 요약 문서 생성

다양한 정보들을 테이블 요약 문서 생성에 활용합니다. 

기본 Schema Description 문서와 Sample Query 등을 모두 활용해서 테이블 요약을 생성합니다. 

아래는 이 정보를 반영하기 위한 LLM 프롬프트 템플릿입니다.

In [5]:
from langchain_aws import ChatBedrock

SYS_PROMPT = """
You are a data analyst that can help summarize SQL tables.
Summarize the provided table by the given context.

<instruction>
- You shall write the summary based only on the provided information, and make it as detailed as possible.
- Note that above sampled queries are only small sample of queries and thus not all possible use of tables are represented, and only some columns in the table are used.
- Do not use any adjective to describe the table. For example, the importance of the table, its comprehensiveness or if it is crucial, or who may be using it. For example, you can say that a table contains certain types of data, but you cannot say that the table contains a 'wealth' of data, or that it is 'comprehensive'.
- Do not mention about the sampled query. Only talk objectively about the type of data the table contains and its possible utilities.
- Please also include some potential usecases of the table, e.g. what kind of questions can be answered by the table, what kind of anlaysis can be done by the table, etc.
- Please provide the output in Korean.
</instruction>
"""

PROMPT_TEMPLATE = """
<table schema>
{table_schema}
</table schema>

<sample queries>
{sample_queries}
</sample queries>
"""

model_kwargs =  { 
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "system": SYS_PROMPT
}

chat_model = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name='us-west-2',
    model_kwargs=model_kwargs
)

In [6]:
def search_table_queries(queries, table_name):  # 테이블이 어떤 쿼리에 사용되었는지 검색하여 추출하는 함수입니다.
    table_name_lower = table_name.lower()
    matched_queries = []

    for line in queries:
        try:
            query_data = json.loads(line)
            if table_name_lower in query_data['query'].lower():
                matched_queries.append(query_data)
        except json.JSONDecodeError:
            print(f"Invalid JSON line: {line}")
    
    return matched_queries

#### 주어진 정보를 바탕으로 `Customer`라는 테이블에 대한 요약 문서를 추출해보겠습니다.

In [13]:
queries

['{"input": "모든 아티스트 정보 조회\\n\\n이 요청은 데이터베이스에 저장된 모든 아티스트의 전체 정보를 확인하기 위한 것입니다. 아티스트 ID와 이름을 포함한 모든 필드를 조회합니다.\\n\\n비즈니스 목적:\\n- 음악 플랫폼이나 갤러리에서 전체 아티스트 목록 확인\\n- 아티스트 관리 시스템에서 등록된 모든 아티스트 정보 검토\\n- 새로운 아티스트 추가 전 기존 아티스트 확인\\n- 아티스트 데이터베이스의 전반적인 상태 점검\\n\\n사용자는 아티스트 관련 작업을 시작하기 전에 현재 시스템에 등록된 모든 아티스트를 파악하고자 할 때 이러한 요청을 할 것입니다. 특히 관리자나 데이터 분석가가 아티스트 데이터 전체를 검토하거나 내보내기 위해 사용할 수 있습니다.", "query": "SELECT * FROM Artist"}\n',
 '{"input": "AC/DC 아티스트가 발매한 모든 앨범 정보\\n\\n이 쿼리는 특정 아티스트(AC/DC)가 발매한 모든 앨범의 상세 정보를 조회하는 것입니다. 음악 스트리밍 서비스나 음반 판매점에서 특정 아티스트의 전체 디스코그래피를 확인하려는 목적으로 사용됩니다. 사용자가 좋아하는 아티스트의 모든 작품을 한눈에 보거나, 컬렉션을 완성하기 위해 아직 구매하지 않은 앨범을 확인하려는 의도가 있습니다. 또한 음악 플랫폼에서 아티스트 페이지를 구성할 때 해당 아티스트의 모든 앨범을 표시하기 위한 백엔드 쿼리로도 활용될 수 있습니다.", "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = \'AC/DC\')"}\n',
 '{"input": "록 장르에 속하는 모든 음악 트랙 정보\\n\\n이 요청은 음악 데이터베이스에서 특정 장르에 속하는 모든 트랙을 찾고자 하는 목적이 있습니다. 사용자는 록 음악만 선별적으로 보고 싶어하며, 해당 장르의 모든 트랙 정보(제목, 아티스트, 앨범 등)를 한번에 확인하려는 의도가 

In [12]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

table_name = 'Customer'

matched_queries = search_table_queries(queries, table_name)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
chain = prompt | chat_model | StrOutputParser()

table_summary = chain.invoke({"table_schema": schema[0]['Customer'], "sample_queries": matched_queries})
print(table_summary)

# 고객 테이블 요약

이 테이블은 고객의 상세 정보와 담당 지원 담당자에 대한 연결 정보를 포함하고 있습니다. 

## 포함 데이터
- 고객 식별 정보: 고유 식별자(CustomerId)
- 개인 정보: 이름(FirstName), 성(LastName), 이메일(Email), 전화번호(Phone), 팩스번호(Fax)
- 소속 정보: 회사명(Company)
- 주소 정보: 주소(Address), 도시(City), 주/도(State), 국가(Country), 우편번호(PostalCode)
- 관계 정보: 담당 지원 담당자 ID(SupportRepId)

## 활용 가능성
1. **고객 분석**:
   - 지역별 고객 분포 분석
   - 국가별 고객 수 파악
   - 특정 지역 고객 타겟팅을 위한 데이터 추출

2. **고객 관계 관리**:
   - 고객별 담당 지원 담당자 확인
   - 고객 연락처 정보 관리
   - 고객 프로필 구축 및 관리

3. **마케팅 활동**:
   - 지역 기반 마케팅 캠페인 대상 선정
   - 이메일 마케팅을 위한 연락처 추출
   - 특정 회사 소속 고객 대상 B2B 마케팅

4. **비즈니스 인텔리전스**:
   - 고객 데이터와 매출 데이터 연계 분석
   - 고객 세그먼트별 구매 패턴 분석
   - 고객 충성도와 구매 행동 간의 상관관계 분석

5. **고객 서비스 최적화**:
   - 담당자별 고객 할당 현황 파악
   - 지역별 고객 서비스 요구사항 분석
   - 고객 연락 채널 선호도 파악

이 테이블은 고객 관리 시스템의 핵심 구성 요소로, 고객 정보를 체계적으로 관리하고 다양한 비즈니스 의사결정에 필요한 기초 데이터를 제공합니다.


#### 아래는 이 작업을 Schema Description 내 모든 테이블을 대상으로 수행하는 코드입니다 (약 2-3분 소요됩니다)

In [15]:
import os

OUTPUT_FILE_PATH1 = "./chinook_detailed_schema_temp.json"

with open(OUTPUT_FILE_PATH1, 'w', encoding='utf-8') as output_file:
    output_file.write('[\n')

@retry(total_try_cnt=5, sleep_in_sec=10, retryable_exceptions=(ClientError,))
def summarize_table(table_name, table_data, queries, chain):
    table_summary = chain.invoke({"table_schema": table_data, "sample_queries": queries})
    table_data['table_summary'] = table_summary 
    summary_output = {table_name: table_data}
    return summary_output
    
for table_info in schema:
    for table_name, table_data in table_info.items():
        globals()[table_name] = table_data
        matched_queries = search_table_queries(queries, table_name)
        prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        chain = prompt | chat_model | StrOutputParser()

        table_summary = summarize_table(table_name, table_data, matched_queries, chain)
        
        with open(OUTPUT_FILE_PATH1, 'a', encoding='utf-8') as output_file:
            output_file.write(json.dumps(table_summary, ensure_ascii=False, indent=4) + ',\n')

with open(OUTPUT_FILE_PATH1, 'rb+') as output_file:
    output_file.seek(-2, os.SEEK_END) 
    output_file.truncate() 
    output_file.write(b'\n]')

INFO:retry-bedrock-invocation:trying summarize_table() [1/5]
INFO:retry-bedrock-invocation:in retry(), summarize_table() returned '{'Album': {'table_desc': 'Stores album data with unique ID, title, and links to artist via artist ID.', 'cols': [{'col': 'AlbumId', 'col_desc': 'Primary key, unique identifier for the album.'}, {'col': 'Title', 'col_desc': 'Title of the album.'}, {'col': 'ArtistId', 'col_desc': 'Foreign key that references the artist of the album.'}], 'table_summary': '# Album 테이블 요약\n\nAlbum 테이블은 음악 앨범 정보를 저장하는 데이터베이스 테이블입니다. 이 테이블은 다음과 같은 주요 컬럼으로 구성되어 있습니다:\n\n1. **AlbumId**: 앨범의 고유 식별자로 기본 키(Primary Key) 역할을 합니다.\n2. **Title**: 앨범의 제목을 저장합니다.\n3. **ArtistId**: 앨범을 발매한 아티스트를 참조하는 외래 키(Foreign Key)입니다.\n\n## 활용 사례\n\n이 테이블은 다음과 같은 다양한 용도로 활용될 수 있습니다:\n\n1. **아티스트별 디스코그래피 조회**: 특정 아티스트가 발매한 모든 앨범 목록을 조회할 수 있습니다.\n2. **앨범 카탈로그 관리**: 음악 라이브러리나 스트리밍 서비스에서 앨범 정보를 체계적으로 관리할 수 있습니다.\n3. **앨범-아티스트 연관 분석**: 아티스트와 그들의 앨범 간의 관계를 분석할 수 있습니다.\n4. **앨범 검색 기능**: 제목이나 아티스트 ID를 기준으로 앨범을 검색

이제 `text-to-sql-bedrock/lab3_text2sql_schema_preparation/chinook_detailed_schema_temp.json` 파일을 열어보면, table_summary가 스키마 문서에 추가되어 있습니다.

위와 같이, 1) 테이블에 어떤 컬럼들이 있는지, 2) 어떤 용도로 활용되는지에 대한 자세한 정보를 LLM에 전달하는 것은 올바른 테이블 선택에 도움이 됩니다.

하지만, 테이블 요약이 너무 길어졌을 때 모든 테이블의 요약 정보를 LLM에 전달할 수 없으므로, 테이블 요약 정보 역시 벡터 유사도 검색으로 탐색하는 것이 좋습니다.

## Step 3: 테이블 요약 문서를 벡터 임베딩으로 변환하여 OpenSearch에 저장

이 Step은 `1.sample_queries.ipynb`에서 수행한 샘플 쿼리 저장과정과 유사하게 진행됩니다.

In [16]:
import yaml
from opensearchpy import OpenSearch, RequestsHttpConnection
INDEX_NAME = "schema_description"

def load_opensearch_config():
    with open("../libs/opensearch.yml", 'r', encoding='utf-8') as file:
        return yaml.safe_load(file)

def init_opensearch(config):
    mapping = {"settings": config['settings'], "mappings": config['mappings-detailed-schema']}
    endpoint = opensearch_domain_endpoint
    http_auth = (opensearch_user_id, opensearch_user_password)

    os_client = OpenSearch(
            hosts=[{'host': endpoint.replace("https://", ""),'port': 443}],
            http_auth=http_auth, 
            use_ssl=True,
            verify_certs=True,
            timeout=300,
            connection_class=RequestsHttpConnection
    )

    create_os_index(os_client, mapping)
    return os_client

def create_os_index(os_client, mapping):
    exists = os_client.indices.exists(INDEX_NAME)

    if exists:
        os_client.indices.delete(index=INDEX_NAME)
        print("Existing index has been deleted. Create new one.")
    else:
        print("Index does not exist, Create one.")

    os_client.indices.create(INDEX_NAME, body=mapping)

config = load_opensearch_config()
os_client = init_opensearch(config)

Index does not exist, Create one.


In [17]:
from langchain_community.embeddings import BedrockEmbeddings

emb_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", region_name='us-west-2', model_kwargs={"dimensions":1024}) 
OUTPUT_FILE_PATH2 = "./chinook_detailed_schema.json"

def embedding_summary(emb_model):
    with open(OUTPUT_FILE_PATH1, 'r', encoding='utf-8') as input_file:
        data_list = json.load(input_file)

    for data in data_list:
        table_name = list(data.keys())[0]
        table_summary = data[table_name]["table_summary"]
        data[table_name]["table_summary_v"] = emb_model.embed_query(table_summary)
    
    with open(OUTPUT_FILE_PATH2, 'w', encoding='utf-8') as output_file:
        json.dump(data_list, output_file, ensure_ascii=False, indent=4)

embedding_summary(emb_model)

/tmp/ipykernel_10391/912569524.py:3: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  emb_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", region_name='us-west-2', model_kwargs={"dimensions":1024})


#### 이제 `text-to-sql-bedrock/lab3_text2sql_schema_preparation/chinook_detailed_schema_temp.json` 파일을 열어보면, 
#### table_summary 및 이에 대한 임베딩이 스키마 문서에 추가되어 있습니다.

In [18]:
def load_detailed_schema_descriptions(os_client):

    with open(OUTPUT_FILE_PATH2, 'r') as file:
        schema_data = json.load(file)

    bulk_data = []
    for table in schema_data:
        for table_name, table_info in table.items():
            table_doc = {
                "table_name": table_name,
                "table_desc": table_info["table_desc"],
                "columns": [{"col_name": col["col"], "col_desc": col["col_desc"]} for col in table_info["cols"]],
                "table_summary": table_info["table_summary"],
                "table_summary_v": table_info["table_summary_v"]
            }
            bulk_data.append({"index": {"_index": INDEX_NAME, "_id": table_name}})
            bulk_data.append(table_doc)
    
    bulk_data_str = '\n'.join(json.dumps(item) for item in bulk_data) + '\n'

    response = os_client.bulk(body=bulk_data_str)
    if response["errors"]:
        print("There were errors during bulk indexing:")
        for item in response["items"]:
            if 'index' in item and item['index']['status'] >= 400:
                print(f"Error: {item['index']['error']['reason']}")
    else:
        print("Bulk-inserted all items successfully.")

load_detailed_schema_descriptions(os_client)

Bulk-inserted all items successfully.


#### 이제 OpenSearch에 스키마 정보의 저장을 완료했습니다.